# Libraries

In [ ]:
import requests 
import pandas as pd
import numpy as np
import datetime
import time
#from geopy import *
from math import *
from xml.etree.ElementTree import parse
from xml.etree.ElementTree import fromstring
from datetime import timedelta

In [ ]:
def first(doc):
  
  #rows and column for MonitoredStopVisit object
  colsMonitoredStopVisit = ['RecordedAtTime', 'ItemIdentifier', 'MonitoringRef']
  rowsMonitoredStopVisit = []

  #rows and column for MonitoredVehicleJourney object
  colsMonitoredVehicleJourney = ['LineRef', 'DirectionRef', 'PublishedLineName','OperatorRef','DestinationRef','OriginAimedDepartureTime',
                                'Bearing','Velocity','VehicleRef']
  rowsMonitoredVehicleJourney = []

  #rows and column for VehicleLocation object
  colsVehicleLocation = ['Longitude', 'Latitude']
  rowsVehicleLocation = []

  #rows and column for MonitoredCall object
  colsMonitoredCall = ['StopPointRef', 'Order','ExpectedArrivalTime','DistanceFromStop']
  rowsMonitoredCall = []

  for document in doc.iterfind('ServiceDelivery'):
    for item in document.iterfind('StopMonitoringDelivery'):
      print(item.findtext('ResponseTimestamp'))
      print(item.findtext('Status'))
      for item2 in item.iterfind('MonitoredStopVisit'):
        rowsMonitoredStopVisit.append({
                    'RecordedAtTime': item2.findtext('RecordedAtTime'), 
                    'ItemIdentifier': item2.findtext('ItemIdentifier'),
                    'MonitoringRef':item2.findtext('MonitoringRef')})
        dfMonitoredStopVisit = pd.DataFrame(rowsMonitoredStopVisit, columns = colsMonitoredStopVisit) 
        for item3 in item2.iterfind('MonitoredVehicleJourney'):
          rowsMonitoredVehicleJourney.append({
                    'LineRef': item3.findtext('LineRef'), 
                    'DirectionRef': item3.findtext('DirectionRef'),
                    'PublishedLineName':item3.findtext('PublishedLineName'),
                    'OperatorRef':item3.findtext('OperatorRef'),
                    'DestinationRef':item3.findtext('DestinationRef'),
                    'OriginAimedDepartureTime':item3.findtext('OriginAimedDepartureTime'),
                    'Bearing':item3.findtext('Bearing'),
                    'Velocity':item3.findtext('Velocity'),
                    'VehicleRef':item3.findtext('VehicleRef')})      
          dfMonitoredVehicleJourney = pd.DataFrame(rowsMonitoredVehicleJourney, columns = colsMonitoredVehicleJourney) 
          for item4 in item3.iterfind('VehicleLocation'):
            rowsVehicleLocation.append({
                    'Longitude': item4.findtext('Longitude'), 
                    'Latitude': item4.findtext('Latitude')})      
            dfrowsVehicleLocation = pd.DataFrame(rowsVehicleLocation, columns = colsVehicleLocation)
          for item5 in item3.iterfind('MonitoredCall'):
            rowsMonitoredCall.append({
                    'StopPointRef': item5.findtext('StopPointRef'), 
                    'Order': item5.findtext('Order'),
                    'ExpectedArrivalTime':item5.findtext('ExpectedArrivalTime'),
                    'DistanceFromStop':item5.findtext('DistanceFromStop')})      
            dfMonitoredCall = pd.DataFrame(rowsMonitoredCall, columns = colsMonitoredCall) 

In [ ]:
def secondFunc(doc):
    
  i=1
  #all cols and rows
  colsAll = ['RecordedAtTime', 'ItemIdentifier', 'MonitoringRef','LineRef', 'DirectionRef', 'PublishedLineName','OperatorRef','DestinationRef','OriginAimedDepartureTime',
                                'Bearing','Velocity','VehicleRef','Latitude','Longitude','StopPointRef', 'Order','ExpectedArrivalTime','DistanceFromStop']
  rowsAll = []
  for document in doc.iterfind('ServiceDelivery'):
    for item in document.iterfind('StopMonitoringDelivery'):
      #print(item.findtext('ResponseTimestamp'))
      #print(item.findtext('Status'))
      for item2 in item.iterfind('MonitoredStopVisit'):
        for item3 in item2.iterfind('MonitoredVehicleJourney'):   
          for item4 in item3.iterfind('VehicleLocation'):
            i = i+1
            for item5 in item3.iterfind('MonitoredCall'):
              if item3.findtext('PublishedLineName') == '921':
                rowsAll.append({
                      'RecordedAtTime': item2.findtext('RecordedAtTime'), 
                      'ItemIdentifier': item2.findtext('ItemIdentifier'),
                      'MonitoringRef':item2.findtext('MonitoringRef'),
                      'LineRef': item3.findtext('LineRef'), 
                      'DirectionRef': item3.findtext('DirectionRef'),
                      'PublishedLineName':item3.findtext('PublishedLineName'),
                      'OperatorRef':item3.findtext('OperatorRef'),
                      'DestinationRef':item3.findtext('DestinationRef'),
                      'OriginAimedDepartureTime':item3.findtext('OriginAimedDepartureTime'),
                      'Bearing':item3.findtext('Bearing'),
                      'Velocity':item3.findtext('Velocity'),
                      'VehicleRef':item3.findtext('VehicleRef'),
                      'Longitude': item4.findtext('Longitude'), 
                      'Latitude': item4.findtext('Latitude'),
                      'StopPointRef': item5.findtext('StopPointRef'), 
                      'Order': item5.findtext('Order'),
                      'ExpectedArrivalTime':item5.findtext('ExpectedArrivalTime'),
                      'DistanceFromStop':item5.findtext('DistanceFromStop')})      
                #dfMonitoredCall = pd.DataFrame(rowsMonitoredCall, columns = colsMonitoredCall) 
            dfAll = pd.DataFrame(rowsAll, columns = colsAll)
    return dfAll

getting the xml and exporting it an Excel file :  

this for loop is set to run 2000 times a day, and collect the  XML file every 15 seconds - with 1 seconds dely using sleep function.

 using http request we are getting out xml script from ministry of transportation under unique id and password,

using the http request from the request library, we pull the data in real time, activating the second function, not before we take out of the given xml defult line, which makes the function less practical. 
then, the pulled data is saved into temp dataframe, then its added to the final data frame and so on, in each iteration.

In [ ]:
for i in range(2000):
    #document = parse('transport2.xml')
    # url of rss feed 
    xml='http://moran.mot.gov.il:110/Channels/HTTPChannel/SmQuery/2.8/xml?Key=YP1451616&MonitoringRef=42658,42362,40964,40958,40897,40852,42286,40851,41457,41621,41643,40850,42963&PublishedLineName=921'   
    #xml='http://moran.mot.gov.il:110/Channels/HTTPChannel/SmQuery/2.8/xml?Key=YP1451616&MonitoringRef=42658,42362,40964,40958,40897,40852,42286,40851,41457,41621,41643,40850,42963,40847,40845,40839,41454,40836,40834,40832,40833,40841,46176,46177,47041,46179,46180,40253,40240,40243,41705,40757,40161,40097,40059,40060,41711,40082,42133,40772,40110,47962,40079,48138,30352,30358,32784,30404,32781,30379,30401,30386,30361,30341,39374,39337,30302,36476,34238,34237,33810,34236,37865,37861,37206,32734,37209,37254,39684,38303,32409,39628,34063,32259,37655,27059,20176,21654,23034,21705,21472,28660,23013&PublishedLineName=921'
    resp = requests.get(xml)
    #print(i)
    resp2=resp.text
    resp2=resp2.replace('xmlns="http://www.siri.org.uk/siri"' ,"")
    doc=fromstring(resp2)
    df_f = secondFunc(doc)
    time.sleep(16)
    df = df.append(df_f, ignore_index=True)
    

In [ ]:
df.to_excel('Some_Name.xls')
df = df[0:0]